In [1]:
import telegram
import requests
import telegram_send
import pandas as pd # Para evitar escrever pandas e trocar pela escrita apenas de pd para facilitar
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import yfinance as yf 
import schedule
import time
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import stockstats

from bs4 import BeautifulSoup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from pandas_datareader import data as web # Evita a escrita do data e troca pelo web
from datetime import datetime
from stockstats import StockDataFrame
from plotly.subplots import make_subplots

In [2]:
token = '1796099831:AAGteQJXlcNd2dLSSq0kbbqa0QoV0B_xtTU'
    
bot = telegram.Bot(token) #Replace TOKEN with your token string
updater = Updater(token=token, use_context=True) #Replace TOKEN with your token string

dispatcher = updater.dispatcher

In [3]:
def get_coin_bid(coin):
    response = requests.get('https://economia.awesomeapi.com.br/json/last/'+ coin)
    result = response.json()
    c = coin.replace('-','')
    return result[c]['bid']

In [4]:
def get_bdrs_price(paper):
    response = 'https://statusinvest.com.br/bdrs/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value')).split('>')[1].split('<')[0]
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    var = var.split('\n')[4].split('>')[1].split('<')[0]
        
    if not var.startswith('-'):
        var = '+' + var
            
    return stock_price, var

In [5]:
def get_fiis_price(paper):
    response = 'https://statusinvest.com.br/fundos-imobiliarios/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value'))
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    if stock_price == 'None':
        return get_bdrs_price(paper)
    else: 
        stock_price = stock_price.split('>')[1].split('<')[0]
        var = var.split('\n')[4].split('>')[1].split('<')[0]
        
        if not var.startswith('-'):
            var = '+' + var
            
        return stock_price, var
    

In [6]:
def get_stock_price(paper):
    response = 'https://statusinvest.com.br/acoes/' + paper
    result = requests.get(response)
    soup = BeautifulSoup(result.content, 'html.parser')
    stock_price = str(soup.find(class_='value'))
    var = str(soup.find(title='Variação do valor do ativo com base no dia anterior'))
    
    if stock_price == 'None':
        return get_fiis_price(paper)
    else: 
        
        stock_price = stock_price.split('>')[1].split('<')[0]
        var = var.split('\n')[4].split('>')[1].split('<')[0]
        
        if not var.startswith('-'):
            var = '+' + var
            
        return stock_price, var
    
    
    
    

In [7]:
def stock(update, context):
    paper = " ".join(context.args)
    value = get_stock_price(paper.lower())
    update.message.reply_text(paper.upper() + ": R$" + value[0] + '\n \n Variação do dia: ' + value[1])

In [8]:
def euro(update, context):
    eur = "Cotação do Euro Hoje: R$ " + str(get_coin_bid('EUR-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=eur)

In [9]:
def vilzyn(update, context):
    dol = "Cotação do Dolar Hoje: R$ " + str(get_coin_bid('USD-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=dol)

In [10]:
def dolar(update, context):
    dol = "Cotação do Dolar Hoje: R$ " + str(get_coin_bid('USD-BRL')).replace('.',',')
    context.bot.send_message(chat_id=update.effective_chat.id, text=dol)

In [11]:
def chart(update, context):
    paper = " ".join(context.args)
    get_stock_chart(paper)
    context.bot.sendPhoto(chat_id=update.effective_chat.id, photo=open(paper + '.png', 'rb'))
    os.remove(paper + '.png')

In [12]:
def hey(update, context):
    image = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRowHhdTB04EQSOcRuRTqLHvk96K4lS3UupYg&usqp=CAU"
    context.bot.sendPhoto(chat_id=update.effective_chat.id, photo = image, caption = 'Eai Kamako')

In [65]:
def get_stock_chart(paper):
    data = yf.download(
    tickers=paper + '.SA', 
    period="1w",
    interval="5m")
    # Plot the close prices
    
    data_rsi = yf.download(
    tickers=paper + '.SA', 
    period="2mo",
    interval="1d")
    
    
    rsi = calculate_RSI(data_rsi)
    
    data = data.reset_index()
    data = data.rename(columns={data.columns[0]:'Date'})
    
    fig = make_subplots(rows=2, cols=1, row_heights=[0.6, 0.4])
    fig.add_trace(
        go.Candlestick(x=data['Date'],
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close']),
            row=1, col=1)
    
    fig.add_trace(
        go.Scatter(x=rsi['Date'], y=rsi['RSI']),
            row=2, col=1)
    
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(paper_bgcolor='#000326')
    fig.update_layout(font_color='white')
    fig.update_coloraxes(colorbar_tickfont_color='white')
    fig.update_layout(plot_bgcolor='#000326')
    fig.write_image(paper + '.png')
    fig.show()
    

In [66]:
def calculate_RSI(data_rsi, n=14):
    delta = data_rsi['Adj Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=n-1, adjust=False).mean()
    ema_down = down.ewm(com=n-1, adjust=False).mean()
    rs = ema_up/ema_down
    data_rsi['RSI'] = 100 - (100/(1 + rs))
    data_rsi = data_rsi.iloc[14:]
    data_rsi = data_rsi.reset_index()
    return data_rsi
    

In [67]:
get_stock_chart('amar3')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [24]:
rsi

Date
2021-04-19          NaN
2021-04-20     0.000000
2021-04-22     0.000000
2021-04-23     7.142857
2021-04-26    44.285537
2021-04-27    26.902473
2021-04-28    24.755867
2021-04-29    22.139944
2021-04-30    20.064261
2021-05-03    20.863243
2021-05-04    20.210582
2021-05-05    43.181001
2021-05-06    42.449842
2021-05-07    57.258299
2021-05-10    53.366649
2021-05-11    59.834447
2021-05-12    50.276865
2021-05-13    49.366984
2021-05-14    58.964150
2021-05-17    66.395295
2021-05-18    60.634456
2021-05-19    62.828150
2021-05-20    64.766403
2021-05-21    62.531868
2021-05-24    55.466017
2021-05-25    55.690685
2021-05-26    60.612351
2021-05-27    68.480819
2021-05-28    72.216587
2021-05-31    75.817683
2021-06-01    76.009040
2021-06-02    81.760164
2021-06-04    75.201370
2021-06-07    76.188770
2021-06-08    81.114916
2021-06-09    78.019394
2021-06-10    71.260943
2021-06-11    71.350036
2021-06-14    76.714025
2021-06-15    69.596114
2021-06-16    70.844653
2021-06-17 

In [18]:
dispatcher.add_handler(CommandHandler("hey", hey))
dispatcher.add_handler(CommandHandler('dolar', dolar))
dispatcher.add_handler(CommandHandler('euro', euro))
dispatcher.add_handler(CommandHandler('vilzyn', vilzyn))

dispatcher.add_handler(CommandHandler("stock", stock))
dispatcher.add_handler(CommandHandler("chart", chart))

In [19]:
updater.start_polling()